# HW3 Problem 1: Heat Cascade Transshipment Problem

Objective $$\min Q_s + Q_w$$

Subject to heat balance constraints (inflow = outflow)

$$
\begin{aligned}
    Q_s &= 30 + R_1\\
    R_1 + 60 &= 90 + R_2\\
    R_2 + 480 &= R_3 + 357\\
    R_3 + 180 &= 78 + Q_w
\end{aligned}
$$

and the nonnegativity constraints

$$Q_s, Q_w, R_1, R_2, R_3 \ge 0$$

In [2]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

# Hot supplies into intervals (MW)
H1_to_2 = 60
H1_to_3 = 160
H1_to_4 = 60
H2_to_3 = 320
H2_to_4 = 120

# Cold demands from intervals to sinks (MW)
I1_to_C1 = 30
I2_to_C1 = 90
I3_to_C1 = 240
I3_to_C2 = 117
I4_to_C2 = 78

# Variables
m.Qs = pyo.Var(domain=pyo.NonNegativeReals)  # steam heating into interval 1
m.Qw = pyo.Var(domain=pyo.NonNegativeReals)  # cooling water out of interval 4

m.R1 = pyo.Var(domain=pyo.NonNegativeReals)  # residual 1->2
m.R2 = pyo.Var(domain=pyo.NonNegativeReals)  # residual 2->3
m.R3 = pyo.Var(domain=pyo.NonNegativeReals)  # residual 3->4

# Objective
m.obj = pyo.Objective(expr=m.Qs + m.Qw, sense=pyo.minimize)

# Constraints
# 1. Interval 1: Qs = (to C1) + R1
m.bal1 = pyo.Constraint(expr=m.Qs == I1_to_C1 + m.R1)

# 2. Interval 2: R1 + H1->2 = (to C1) + R2
m.bal2 = pyo.Constraint(expr=m.R1 + H1_to_2 == I2_to_C1 + m.R2)

# 3. Interval 3: R2 + (H1->3 + H2->3) = (to C1 + to C2) + R3
m.bal3 = pyo.Constraint(
    expr=m.R2 + (H1_to_3 + H2_to_3) == (I3_to_C1 + I3_to_C2) + m.R3
)

# 4. Interval 4: R3 + (H1->4 + H2->4) = (to C2) + Qw
m.bal4 = pyo.Constraint(
    expr=m.R3 + (H1_to_4 + H2_to_4) == I4_to_C2 + m.Qw
)

# Solve
solver = pyo.SolverFactory("gurobi", solver_io="python")
results = solver.solve(m, tee=False)

# Display results
print("\n--- Optimal values (MW) ---")
print(f"Qs = {pyo.value(m.Qs):.6g}")
print(f"Qw = {pyo.value(m.Qw):.6g}")
print(f"R1 = {pyo.value(m.R1):.6g}")
print(f"R2 = {pyo.value(m.R2):.6g}")
print(f"R3 = {pyo.value(m.R3):.6g}")
print(f"Objective Qs+Qw = {pyo.value(m.obj):.6g}")



--- Optimal values (MW) ---
Qs = 60
Qw = 225
R1 = 30
R2 = 0
R3 = 123
Objective Qs+Qw = 285
